In [1]:
# 安装 TensorFlow

import tensorflow as tf

载入并准备好 `MNIST` 数据集。将样本从整数转换为浮点数：

In [2]:
mnist = tf.keras.datasets.mnist

In [3]:
mnist

<module 'tensorflow.python.keras.api._v1.keras.datasets.mnist' from '/home/wbb/下载/pythonProject/venv/lib/python3.6/site-packages/tensorflow_core/python/keras/api/_v1/keras/datasets/mnist/__init__.py'>

In [4]:
dir(mnist)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_module_wrapper',
 '_sys',
 'load_data']

In [5]:
help(mnist.load_data)

Help on function load_data in module tensorflow.python.keras.datasets.mnist:

load_data(path='mnist.npz')
    Loads the MNIST dataset.
    
    Arguments:
        path: path where to cache the dataset locally
            (relative to ~/.keras/datasets).
    
    Returns:
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    
    License:
        Yann LeCun and Corinna Cortes hold the copyright of MNIST dataset,
        which is a derivative work from original NIST datasets.
        MNIST dataset is made available under the terms of the
        [Creative Commons Attribution-Share Alike 3.0 license.](
        https://creativecommons.org/licenses/by-sa/3.0/)



In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
x_train
x_train.shape

(60000, 28, 28)

In [8]:
y_train.shape

(60000,)

In [9]:
x_train, x_test = x_train / 255.0, x_test / 255.0

将模型的各层堆叠起来，以搭建 [tf.keras.Sequential](https://tensorflow.google.cn/api_docs/python/tf/keras/Sequential?hl=zh_cn) 模型。为训练选择优化器和损失函数：

In [10]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


训练并验证模型：

# 使用 TensorBoard

当使用 Keras's `Model.fit()` 函数进行训练时, 添加 `tf.keras.callback.TensorBoard` 回调可确保创建和存储日志.另外，在每个时期启用 `histogram_freq=1` 的直方图计算功能（默认情况下处于关闭状态）

将日志放在带有时间戳的子目录中，以便轻松选择不同的训练运行。

In [11]:
import datetime

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [12]:
# # train
# model.fit(x_train, y_train, epochs=5)
# # test
# model.evaluate(x_test,  y_test, verbose=2)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 11s 188us/sample - loss: 0.3019 - acc: 0.9118 - val_loss: 0.1480 - val_acc: 0.9545
Epoch 2/5
60000/60000 [==============================] - 13s 210us/sample - loss: 0.1441 - acc: 0.9573 - val_loss: 0.1072 - val_acc: 0.9675
Epoch 3/5
60000/60000 [==============================] - 15s 244us/sample - loss: 0.1081 - acc: 0.9677 - val_loss: 0.0871 - val_acc: 0.9736
Epoch 4/5
60000/60000 [==============================] - 14s 236us/sample - loss: 0.0882 - acc: 0.9731 - val_loss: 0.0859 - val_acc: 0.9741
Epoch 5/5
60000/60000 [==============================] - 10s 170us/sample - loss: 0.0743 - acc: 0.9768 - val_loss: 0.0789 - val_acc: 0.9748


通过命令行 （command） 或在 notebook 中启动 TensorBoard ，这两个接口通常是相同的。 在 notebooks, 使用 `%tensorboard `命令。 在命令行中， 运行不带“％”的相同命令。

In [13]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

简要概述所显示的仪表板（顶部导航栏中的选项卡）：

- Scalars 显示损失和指标在每个时期如何变化。 您还可以使用它来跟踪训练速度，学习率和其他标量值。
- Graphs 可帮助您可视化模型。 在这种情况下，将显示层的Keras图，这可以帮助您确保正确构建。
- Distributions 和 Histograms 显示张量随时间的分布。 这对于可视化权重和偏差并验证它们是否以预期的方式变化很有用。

当您记录其他类型的数据时，会自动启用其他 TensorBoard 插件。 例如，使用 Keras TensorBoard 回调还可以记录图像和嵌入。 您可以通过单击右上角的“inactive”下拉列表来查看 TensorBoard 中还有哪些其他插件。